In [1]:
import pandas as pd
import numpy as np
import quandl as qd

In [2]:
qd.ApiConfig.api_key = 'xrJsb6qPU-whbsegBjyE'

In [3]:
airlines = ['DAL','AAL','JBLU','LUV','SAVE','ALK','UAL']
banks = ['C','BAC','MS','JPM','GS','WFC','RY','TD','BNC']
casinos = ['MGM','WYNN','MLCO','BYD','LVS','CZR','ERI','PENN']

In [20]:
df = qd.get_table('SHARADAR/SF1',dimension='MRY',ticker='UAL')


In [7]:
for col in df.columns:
    print(col)

ticker
dimension
calendardate
datekey
reportperiod
lastupdated
accoci
assets
assetsavg
assetsc
assetsnc
assetturnover
bvps
capex
cashneq
cashnequsd
cor
consolinc
currentratio
de
debt
debtc
debtnc
debtusd
deferredrev
depamor
deposits
divyield
dps
ebit
ebitda
ebitdamargin
ebitdausd
ebitusd
ebt
eps
epsdil
epsusd
equity
equityavg
equityusd
ev
evebit
evebitda
fcf
fcfps
fxusd
gp
grossmargin
intangibles
intexp
invcap
invcapavg
inventory
investments
investmentsc
investmentsnc
liabilities
liabilitiesc
liabilitiesnc
marketcap
ncf
ncfbus
ncfcommon
ncfdebt
ncfdiv
ncff
ncfi
ncfinv
ncfo
ncfx
netinc
netinccmn
netinccmnusd
netincdis
netincnci
netmargin
opex
opinc
payables
payoutratio
pb
pe
pe1
ppnenet
prefdivis
price
ps
ps1
receivables
retearn
revenue
revenueusd
rnd
roa
roe
roic
ros
sbcomp
sgna
sharefactor
sharesbas
shareswa
shareswadil
sps
tangibles
taxassets
taxexp
taxliabilities
tbvps
workingcapital


In [27]:
def analyze_debt(stock):
    df = qd.get_table('SHARADAR/SF1',dimension='MRY',ticker=stock)
    debt_cols = ['intexp','cashneq','assetsc','liabilitiesc','ncfdebt','reportperiod','lastupdated']
    debt = (df.T.loc[debt_cols]).T
    debt['quick_ratio'] = debt['cashneq']/debt['liabilitiesc']
    debt['current_ratio'] = debt['assetsc'] / debt['liabilitiesc']
    debt['cash_intexp'] = debt['cashneq'] / debt['intexp']
    debt['cash_ncfdebt'] = debt['cashneq'] / debt['ncfdebt']
    return debt



In [28]:
for stock in airlines:
    print(stock)
    print(analyze_debt(stock))

DAL
         intexp     cashneq     assetsc liabilitiesc      ncfdebt  \
None                                                                
0     301000000  2882000000  8249000000  20204000000    487000000   
1     311000000  1565000000  6340000000  18578000000    693000000   
2     396000000  1814000000  7804000000  18959000000   1196000000   
3     388000000  2762000000  7451000000  15239000000  -1259000000   
4     481000000  1972000000  9056000000  17526000000  -1520000000   

     reportperiod lastupdated quick_ratio cash_intexp current_ratio  \
None                                                                  
0      2019-12-31  2020-02-13    0.142645     9.57475      0.408285   
1      2018-12-31  2020-02-13   0.0842394     5.03215      0.341264   
2      2017-12-31  2020-02-13   0.0956802     4.58081      0.411625   
3      2016-12-31  2020-02-13    0.181245     7.11856      0.488943   
4      2015-12-31  2020-02-13    0.112519     4.09979      0.516718   

     cash_ncfd

In [33]:
analyze_debt('SAVE')

,intexp,cashneq,assetsc,liabilitiesc,ncfdebt,reportperiod,lastupdated,quick_ratio,cash_intexp,current_ratio,cash_ncfdebt
None,,,,,,,,,,,
0,101350000,978957000,1385443000,1112044000,-120348000,2019-12-31,2020-02-06,0.880322,9.65917,1.24585,-8.13439
1,83777000,1004733000,1345466000,834535000,481739000,2018-12-31,2020-02-06,1.20394,11.9929,1.61223,2.08564
2,57302000,800849000,1282110000,664333000,513247000,2017-12-31,2020-02-06,1.20549,13.9759,1.92992,1.56036
3,41654000,700900000,975845000,531950000,352747000,2016-12-31,2020-02-06,1.31761,16.8267,1.83447,1.98698
4,20382000,803632000,1026340000,466240000,495224000,2015-12-31,2020-02-06,1.72364,39.4285,2.20131,1.62276
